## Imports

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Global Variables

In [2]:
frequency = 1
t_cost = 0
target_vol = 0.10
min_regions = 4

## Data

### Riskfree Data

In [3]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_8117/2677495952.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Factor Data

In [4]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_8117/297690613.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745,0.0695
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789,-0.0132
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958,-0.1181
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048,0.0574
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118,0.0618
...,...,...,...,...,...,...,...
2025-05-31,0.0606,-0.0072,-0.0288,0.0129,0.0251,0.0221,0.0256
2025-06-30,0.0486,-0.0002,-0.0160,-0.0320,0.0145,-0.0264,0.0527
2025-07-31,0.0198,-0.0015,-0.0127,-0.0029,-0.0208,-0.0096,0.0184


### Benchmark Data

In [5]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_return = benchmark_data[['MSCI World']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.013121
2025-08-31    0.026408
2025-09-30    0.032574
2025-10-31    0.020226
2025-11-30    0.003149
Freq: ME, Name: MSCI World, Length: 468, dtype: float64

### Macro Data

In [6]:
# --- Load Macro Data ---
CPI_forecasts = pd.read_excel(
    f"{path}/Inflation_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
CPI_forecasts.index = pd.to_datetime(CPI_forecasts.index)
CPI_forecasts.index = CPI_forecasts.index + pd.offsets.MonthEnd(0)
CPI_forecasts *= 100

CPI_forecasts

,UK,CH,JP,AU,EU,US,EM
Date,,,,,,,
1970-01-31,4.95739,2.31338,7.75127,NaN,NaN,NaN,NaN
1970-02-28,4.93065,2.12360,7.75127,NaN,NaN,NaN,NaN
1970-03-31,5.14198,2.49918,7.71710,NaN,NaN,NaN,NaN
1970-04-30,5.61884,2.59281,7.68362,NaN,NaN,NaN,NaN
1970-05-31,6.08365,3.13683,7.24558,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,4.10000,0.10000,3.30000,2.1,1.96760,3.17,-0.07462
2025-07-31,4.20000,0.20000,3.10000,3.2,2.00791,2.79,-0.07462
2025-08-31,4.10000,0.20000,2.70000,3.2,2.02889,2.79,-0.07462


In [7]:
RGDP_forecasts = pd.read_excel(
    f"{path}/RGDP_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
RGDP_forecasts.index = pd.to_datetime(RGDP_forecasts.index)
RGDP_forecasts.index = RGDP_forecasts.index + pd.offsets.MonthEnd(0)
RGDP_forecasts *= 100

RGDP_forecasts

,AU,UK,CH,JP,EU,US,EM
Date,,,,,,,
1970-07-31,3.15789,NaN,NaN,NaN,NaN,2.936175,NaN
1970-08-31,3.15789,NaN,NaN,NaN,NaN,2.936175,NaN
1970-09-30,3.15789,NaN,NaN,NaN,NaN,2.936175,NaN
1970-10-31,5.26317,NaN,NaN,NaN,NaN,2.994250,NaN
1970-11-30,5.26317,NaN,NaN,NaN,NaN,2.994250,NaN
...,...,...,...,...,...,...,...
2025-06-30,2.10000,2.5,2.439516,1.976375,1.522007,1.345350,4.977357
2025-07-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175,4.977357
2025-08-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175,4.977357


### Bond Data

In [8]:
# --- Load Bond Futures ---
bond_futures = pd.read_excel(
    f"{path}/Bond Futures.xlsx",
    index_col = 0,
    parse_dates = True
)
bond_futures.index = pd.to_datetime(bond_futures.index)
bond_futures.index = bond_futures.index + pd.offsets.MonthEnd(0)

bond_futures

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,72.58774,89.88,83.91110,58.968750,90.12,NaN,89.28422
1998-02-28,73.84924,90.45,83.86650,58.437500,91.37,NaN,89.20823
1998-03-31,72.76451,90.75,84.08059,58.281250,90.54,NaN,92.70965
1998-04-30,74.66254,91.56,83.99135,58.250000,89.35,NaN,92.89550
1998-05-31,75.80680,93.14,84.46878,58.593750,91.27,NaN,91.67311
...,...,...,...,...,...,...,...
2025-07-31,147.00613,137.03,61.55103,111.078125,196.99,15.03735,121.79588
2025-08-31,150.42250,136.54,62.63541,112.468750,200.93,15.12679,122.26090
2025-09-30,151.15975,135.79,63.31735,112.484375,205.32,15.14941,122.26156


In [9]:
bond_returns = bond_futures.pct_change()
bond_returns

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,NaN,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,NaN,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,NaN,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,NaN,-0.013159
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


## Signal Generation

In [10]:
# --- Compute Business Cycle Signal ---
CPI_component = CPI_forecasts.diff(12)
RGDP_component = RGDP_forecasts.diff(12)
business_cyle_signal = - (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal = business_cyle_signal.resample('ME').last()
business_cyle_signal = business_cyle_signal.dropna(how='all')
business_cyle_signal.tail()

,AU,CH,EM,EU,JP,UK,US
Date,,,,,,,
2025-06-30,1.71445,-0.564448,0.134685,-0.224851,-1.728625,-1.008965,0.069925
2025-07-31,-0.39146,0.867678,0.280295,0.092139,-0.488158,-1.438370,-0.026338
2025-08-31,-0.39146,0.752568,0.327975,-0.131066,-0.188158,-1.354405,-0.026338
2025-09-30,-0.39146,0.643893,0.230695,-0.431266,-0.538158,-1.593315,-0.026338
2025-10-31,-1.37530,0.606678,0.182500,-0.055999,-0.325256,-1.275500,-0.061275


In [11]:
# --- Compute Weights ---
weights, scaling_factors, weights_raw = pmp.make_country_weights_ls_vol(
    signal=business_cyle_signal,
    returns=bond_returns,
    min_regions=min_regions,
    signal_lag=0,
    vol_target=target_vol,
    vol_lookback=36
)

In [12]:
# Standardized weights (Just for debugging)
weights_raw

,AU,CH,EM,EU,JP,UK,US
1998-01-31,-0.632456,1.264911,NaN,0.632456,-1.264911,NaN,0.000000
1998-02-28,-0.632456,1.264911,NaN,0.632456,-1.264911,NaN,0.000000
1998-03-31,-1.264911,1.264911,NaN,0.632456,-0.632456,NaN,0.000000
1998-04-30,0.632456,1.264911,NaN,0.000000,-1.264911,NaN,-0.632456
1998-05-31,0.632456,1.264911,NaN,0.000000,-1.264911,NaN,-0.632456
...,...,...,...,...,...,...,...
2025-06-30,-1.388730,0.462910,-0.92582,0.000000,1.388730,0.92582,-0.462910
2025-07-31,0.462910,-1.388730,-0.92582,-0.462910,0.925820,1.38873,0.000000
2025-08-31,0.925820,-1.388730,-0.92582,0.000000,0.462910,1.38873,-0.462910
2025-09-30,0.000000,-1.388730,-0.92582,0.462910,0.925820,1.38873,-0.462910


In [13]:
# Scaling factors (NaNs for the first 36 months) (Just for debugging)
scaling_factors

1998-01-31         NaN
1998-02-28         NaN
1998-03-31         NaN
1998-04-30         NaN
1998-05-31         NaN
                ...   
2025-06-30    0.485403
2025-07-31    0.491666
2025-08-31    0.508557
2025-09-30    0.564345
2025-10-31    0.616074
Length: 334, dtype: float64

In [14]:
# Volatility-adjusted weights (Just for debugging)
weights

,AU,CH,EM,EU,JP,UK,US
2000-12-31,0.993311,0.052280,NaN,-0.575075,0.679634,-0.888752,-0.261398
2001-01-31,1.097308,-0.877846,NaN,-0.548654,0.109731,0.438923,-0.219462
2001-02-28,0.882960,-1.103700,NaN,-0.441480,0.220740,0.551850,-0.110370
2001-03-31,0.981507,-0.878191,NaN,-0.568241,-0.258291,0.671558,0.051658
2001-04-30,0.979624,-0.876506,NaN,-0.257796,-0.567151,0.670269,0.051559
...,...,...,...,...,...,...,...
2025-06-30,-0.674093,0.224698,-0.449396,0.000000,0.674093,0.449396,-0.224698
2025-07-31,0.227597,-0.682792,-0.455194,-0.227597,0.455194,0.682792,0.000000
2025-08-31,0.470833,-0.706249,-0.470833,0.000000,0.235416,0.706249,-0.235416
2025-09-30,0.000000,-0.783724,-0.522482,0.261241,0.522482,0.783724,-0.261241


## Backtest

In [15]:
results = pmp.run_cc_strategy(
    weights = weights,
    returns = bond_returns,
    rf = riskfree,
    frequency=1,
    t_cost= t_cost,
    benchmark = benchmark_return
)

results

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2001-01-31,0.032288,0.032288,0.019408,0.000000,0.0,0.0054,0.994572,0.052429,0.000000,-0.571587,0.689924,-0.870792,-0.262257
2001-02-28,-0.003242,-0.003242,-0.084288,1.494324,0.0,0.0038,1.098351,-0.877942,0.000000,-0.545771,0.110880,0.433025,-0.221784
2001-03-31,-0.007300,-0.007300,-0.065527,0.442770,0.0,0.0042,0.882727,-1.120103,0.000000,-0.423396,0.221905,0.542267,-0.110700
2001-04-30,0.007251,0.007251,0.074300,0.628692,0.0,0.0039,0.976015,-0.861359,0.000000,-0.558509,-0.258844,0.659458,0.050489
2001-05-31,-0.004286,-0.004286,-0.012221,0.319828,0.0,0.0032,0.977180,-0.878142,0.000000,-0.244045,-0.572379,0.661593,0.051507
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.001657,0.001657,0.043488,0.011305,0.0,0.0034,-0.690478,0.229995,-0.452738,0.000000,0.676321,0.466166,-0.227608
2025-07-31,-0.007111,-0.007111,0.013121,0.023467,0.0,0.0034,-0.658979,0.221205,-0.445329,0.000000,0.669307,0.429254,-0.222569
2025-08-31,-0.016664,-0.016664,0.026408,1.359127,0.0,0.0038,0.231607,-0.696448,-0.457902,-0.232886,0.453567,0.685399,0.000000


## Performance Summary

In [16]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,8.7759,0.8588
Arithm Avg Xs Return,7.0871,-0.8300
Std Xs Returns,15.4355,10.7880
Sharpe Arithmetic,0.4591,-0.0769
Geom Avg Total Return,7.8407,0.2634
Geom Avg Xs Return,6.1401,-1.4372
Sharpe Geometric,0.3978,-0.1332
Min Xs Return,-19.0140,-17.0223
Max Xs Return,12.8184,8.6013
Skewness,-0.6124,-0.8104


In [17]:
# calculate yearly vol annualized
tmp = results.copy()
tmp["year"] = tmp.index.year

yearly_std = tmp.groupby("year")["ret_net"].std()
yearly_vol_ann = tmp.groupby("year")["ret_net"].std() * (12 ** 0.5)
yearly_vol_ann


year
2001    0.076353
2002    0.090523
2003    0.104224
2004    0.093380
2005    0.060503
2006    0.057400
2007    0.091586
2008    0.277062
2009    0.080349
2010    0.062498
2011    0.117577
2012    0.102473
2013    0.081449
2014    0.052404
2015    0.107914
2016    0.134407
2017    0.083757
2018    0.089527
2019    0.106361
2020    0.115276
2021    0.056571
2022    0.192594
2023    0.102739
2024    0.073889
2025    0.032001
Name: ret_net, dtype: float64